In [3]:
import sk8s

In [17]:
import subprocess
import json
import time
from collections import Counter

In [30]:
def wait(jobs, delete=True):

    def get_job_status_json(jobs):
        stdout = subprocess.run(f"kubectl get jobs {' '.join(jobs)} -o json --ignore-not-found", 
                            check=True, shell=True,
                            stdout=subprocess.PIPE, stderr=subprocess.PIPE, encoding="utf-8").stdout
        try:
            return json.loads(stdout)
        except json.JSONDecodeError:
            return dict(kind="List", items=[])

    def check_job_status_json(data):
        assert(data["kind"] == "Job")
        if ("failed" in data["status"]) and (data["status"]["failed"] >= data["spec"]["backoffLimit"]):
            return "failed"
        if "succeeded" not in data["status"]:
            return "running"
        if data["status"]["succeeded"] == 1:
            return "success"

    def cleanup_job(job_name, delete=True):
        log_text = sk8s.logs(job_name, decode=True)

        print(f"done waiting for {job_name}")

        if delete:
            sk8s.util.run_cmd(f"kubectl delete job {job_name}")

        print(f"cleaned up from {job_name}")

        assert(len(log_text.values()) == 1)
        return list(log_text.values())[0]


    data = get_job_status_json(jobs)
    print(data)
    results = {d["metadata"]["name"]: check_job_status_json(d) for d in data["items"]}
    print(Counter(results.values()))

    while "running" in results.values():
        data = get_job_status_json(jobs)
        results = {d["metadata"]["name"]: check_job_status_json(d) for d in data["items"]}
        print(Counter(results.values()))

        for j, s in results.items():
            if s != "running":
                print(j, s, cleanup_job(j, delete=delete), flush=True)
        time.sleep(5)

    for j, s in results.items():
        if s != "running":
            print(j, s, cleanup_job(jobs[4], delete=delete), flush=True)


jobs = sk8s.map(lambda i: i, range(10), asynchro=True)
time.sleep(1)
wait(jobs)

{'apiVersion': 'v1', 'items': [{'apiVersion': 'batch/v1', 'kind': 'Job', 'metadata': {'annotations': {'batch.kubernetes.io/job-tracking': '', 'kubectl.kubernetes.io/last-applied-configuration': '{"apiVersion":"batch/v1","kind":"Job","metadata":{"annotations":{},"name":"job-rfetu","namespace":"sk8s"},"spec":{"backoffLimit":0,"template":{"spec":{"containers":[{"command":["python","-c","import dill as pickle\\nimport base64\\nimport json\\nimport sys\\nimport sk8s\\n\\nfunc = sk8s.deserialize_func(\\"gASVrAIAAAAAAACMCmRpbGwuX2RpbGyUjBBfY3JlYXRlX2Z1bmN0aW9ulJOUKGgAjAxfY3JlYXRlX2NvZGWUk5QoQwCUSwFLAEsASwFLAksTQwiIAHwAjgBTAJROhZQpjAFhlIWUjChjOlx1c2Vyc1xqYXJlZFxkZXNrdG9wXGpvYnNcc2s4c1xqb2JzLnB5lIwIPGxhbWJkYT6US3xDAggAlIwEZnVuY5SFlCl0lFKUfZSMCF9fbmFtZV9flIwJc2s4cy5qb2JzlHNoCymFlGgAjAxfY3JlYXRlX2NlbGyUk5ROhZRSlIWUdJRSlH2UfZQojA9fX2Fubm90YXRpb25zX1+UfZSMDF9fcXVhbG5hbWVfX5SMFXJ1bi48bG9jYWxzPi48bGFtYmRhPpR1hpRijAhidWlsdGluc5SMB2dldGF0dHKUk5SMBGRpbGyUjAVfZGlsbJSTlIwIX3NldGF0dHKUaCOMB3NldGF0dHKUk5SHl